# GAN Generator Network

In [1]:
import keras
from keras import layers
import numpy as np

latent_dim = 32
height = 32
width = 32
channels = 3

generator_input = keras.Input(shape=(latent_dim,))

x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)                   #3
generator.summary()


/home/jupyter-account/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/jupyter-account/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32768)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 32, 32, 256)       1048832   
__________

# GAN Discriminator Network

In [2]:
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

x = layers.Dropout(0.4)(x)

x = layers.Dense(1, activation='sigmoid')(x)

discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()

discriminator_optimizer = keras.optimizers.RMSprop(
    lr=0.0008,
    clipvalue=1.0,
    decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer,
                     loss='binary_crossentropy')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 6, 6, 128)         0         
__________

# Adversarial Network

In [3]:
discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')


# GAN Training

In [4]:
import os
from keras.preprocessing import image

(x_train, y_train), (_, _) = keras.datasets.cifar10.load_data()

x_train = x_train[y_train.flatten() == 6]

x_train = x_train.reshape(
    (x_train.shape[0],) +
    (height, width, channels)).astype('float32') / 255.

iterations = 10000
batch_size = 20
save_dir = 'DCGAN_example'

start = 0
for step in range(iterations):
    random_latent_vectors = np.random.normal(size=(batch_size,
                                            latent_dim))

    generated_images = generator.predict(random_latent_vectors)

    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])

    labels = np.concatenate([np.ones((batch_size, 1)),
                             np.zeros((batch_size, 1))])
    labels += 0.05 * np.random.random(labels.shape)

    d_loss = discriminator.train_on_batch(combined_images, labels)

    random_latent_vectors = np.random.normal(size=(batch_size,
                                            latent_dim))

    misleading_targets = np.zeros((batch_size, 1))

    a_loss = gan.train_on_batch(random_latent_vectors,
                                misleading_targets)

    start += batch_size
    if start > len(x_train) - batch_size:
      start = 0
    if step % 100 == 0:
        gan.save_weights('gan.h5')

        print(f'{step}/{iterations}:')
        print(f'  discriminator loss: {d_loss}')
        print(f'  adversarial loss: {a_loss}')

        img = image.array_to_img(generated_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir,
                      'generated_frog' + str(step) + '.png'))

        img = image.array_to_img(real_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir,
                      'real_frog' + str(step) + '.png'))


/home/jupyter-account/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0/10000:
  discriminator loss: 0.6826667189598083
  adversarial loss: 0.6764920949935913
100/10000:
  discriminator loss: 0.45859408378601074
  adversarial loss: 1.808218002319336
200/10000:
  discriminator loss: 0.6916437745094299
  adversarial loss: 0.7460620999336243
300/10000:
  discriminator loss: 0.6980230212211609
  adversarial loss: 0.7770370244979858
400/10000:
  discriminator loss: 0.6944787502288818
  adversarial loss: 0.733106255531311
500/10000:
  discriminator loss: 0.6892963647842407
  adversarial loss: 0.7772891521453857
600/10000:
  discriminator loss: 0.7015835046768188
  adversarial loss: 0.7289963960647583
700/10000:
  discriminator loss: 0.6864560842514038
  adversarial loss: 0.7504480481147766
800/10000:
  discriminator loss: 0.687759280204773
  adversarial loss: 0.7427811622619629
900/10000:
  discriminator loss: 0.6957899332046509
  adversarial loss: 0.7666605114936829
1000/10000:
  discriminator loss: 0.7194489240646362
  adversarial loss: 0.75782310962677
1100

9000/10000:
  discriminator loss: 0.6924470067024231
  adversarial loss: 0.8190523982048035
9100/10000:
  discriminator loss: 0.6639738082885742
  adversarial loss: 0.7115256190299988
9200/10000:
  discriminator loss: 0.7090044617652893
  adversarial loss: 0.7607990503311157
9300/10000:
  discriminator loss: 0.6565305590629578
  adversarial loss: 0.8645642995834351
9400/10000:
  discriminator loss: 0.7099415063858032
  adversarial loss: 1.2284138202667236
9500/10000:
  discriminator loss: 0.6501067876815796
  adversarial loss: 0.8638603091239929
9600/10000:
  discriminator loss: 0.6775126457214355
  adversarial loss: 0.8674817085266113
9700/10000:
  discriminator loss: 0.6897228956222534
  adversarial loss: 0.7588996887207031
9800/10000:
  discriminator loss: 0.7194375991821289
  adversarial loss: 0.8644815683364868
9900/10000:
  discriminator loss: 0.6624621748924255
  adversarial loss: 0.7602604031562805
